COCONUT AGE CLASSFICATION

## IMPORTS

In [ ]:
import sys

!{sys.executable} -m pip install opencv-python
!"{sys.executable}" -m pip install opencv-python
%pip install tensorflow
%pip install pandas

'c:\Users\Mary' is not recognized as an internal or external command,
operable program or batch file.


^C



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import cv2
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from tensorflow.keras import layers
from sklearn.linear_model import LinearRegression

## PREPROCESSING

In [ ]:
# direction in which the folder where images are stored
image_dir = 'images' #this  depend kung asa siya gi butang
IMG_SIZE = (224, 224)


# ----------------------------------
# IMAGE PREPROCESSING
#-----------------------------------

img_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),   # Flipping
    layers.RandomRotation(0.2),                     # Rotation
    layers.RandomZoom(0.2),                         # Zooming
    layers.RandomContrast(0.2),                     # Contrast
    layers.RandomTranslation(0.1, 0.1),             # Shifting
    layers.Resizing(224, 224),                      # Resize to MobileNet input size
    layers.Rescaling(1./255, offset=-1),            # Normalization to [-1,1]
    layers.CenterCrop(224, 224)
])
   


In [9]:


# Load the numerical dataset
df = pd.read_csv('Numerical_Datasets.csv', header=1) 


# ----------------------------------
# NUMERICAL PREPROCESSING
#-----------------------------------

# Step 1: Ensure numeric data
for c in ["HeightTrunk", "11leafscars", "Year"]:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Step 2: Applying the PCA Formula: Age = TrunkHeight / L11 + 5
df["age_formula"] = df["HeightTrunk"] / df["11leafscars"] + 5

# Step 3: Min-Max Scaling
if "Year" in df.columns and df["Year"].notna().any():
    df["actual_age"] = 2025 - df["Year"]

    # Step 4: Calibrate the formula using linear regression
    mask = df["actual_age"].notna() & np.isfinite(df["actual_age"])
    x = df.loc[mask, ["age_formula"]]
    y = df.loc[mask, ["actual_age"]]

    # Step 5: Fit linear regression model
    reg = LinearRegression().fit(x, y)
    a = float(reg.coef_[0])
    b = float(reg.intercept_)
else:
    # Fallback if no year column
    a, b = 1.0, 0.0    

# Step 6: Apply calibrated formula
df["final_age"] = a * df["age_formula"] + b

# Steo 7: Clip negative aged or unrealistic ages
df["final_age"] = df["final_age"].clip(lower=0)




# Step 5: Show the results
print("Calibrated a, b:", a, b)
print(df[["age_formula","final_age","actual_age"]].head(200))
#print(df[['HeightTrunk','11leafscars']].head(10))
#print(df[['HeightTrunk','11leafscars']].describe())
#print(df[['age_formula']].head(10))
#print(df[['final_age']].head(10))


Calibrated a, b: 2.7072163520770096 -10.558086853825515
     age_formula  final_age  actual_age
0      20.233333  44.217924          18
1      20.233333  44.217924          18
2      20.233333  44.217924          18
3      19.061538  41.045622          18
4      19.061538  41.045622          18
..           ...        ...         ...
195    25.594595  58.732018          49
196    25.594595  58.732018          49
197    22.720930  50.952387          49
198    22.720930  50.952387          49
199    22.720930  50.952387          49

[200 rows x 3 columns]


C:\Users\Mary Vee\AppData\Local\Temp\ipykernel_23440\1143471018.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  a = float(reg.coef_[0])
C:\Users\Mary Vee\AppData\Local\Temp\ipykernel_23440\1143471018.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  b = float(reg.intercept_)
